# Game theory script #
Author: Andrew Larkin <br>
Developed for Applied Game Theory, Johns Hopkins, Spring 2018 <br>

**Summary** <br>
This script performs multiple repeated games for pro and anti climate change actors who can interact with media that are pro- and anti-climate change regulation.  Models are run for each state using Yale climate communication estimates of party-stratified support for regulation (http://www.pewforum.org/religious-landscape-study/compare/party-affiliation/by/state/) and Pew Internet Reserach center estimates of party affiliation (http://www.pewforum.org/religious-landscape-study/compare/party-affiliation/by/state).


### Import Modules and define filepaths ###

In [ ]:
from copy import deepcopy
import numpy as np
import pandas as ps

### define filepaths, constants, and load data ###

In [ ]:
Parent_Folder = "D:/Dropbox/Dropbox/GameTheory/Project/"
Party_Climate_Support_CSV = Parent_Folder + "ClimateChangePercents.csv" # Yale 2016 Climate Change Opinion Estimates
Party_Fractions_CSV = Parent_Folder + "PartyAffiliationFractions.csv" # Pew Reserach Center 2014 State Party Affiliations

Climate_Support = ps.read_csv(Party_Climate_Support_CSV)
Party_Fractions = ps.read_csv(Party_Fractions_CSV)

OUTCOME_LIST = ['regulate','fundrenewables','happening','human','harmUS']

#Climate_Support.head()
Party_Fractions.head()

### restructure input data into dictionaries for each political party ###

**Inputs: ** <br>
- In_Climate_Dataframe (dictionary) - pandas dataframe containing Yale Climate Change opinion data
- In_Fraction_Dataframe (dictionary) - game number in a multi game sequence

**Outputs: ** <br>
- Dict_Dict (dictionary) - dictionary containing subdictionaries for each party affiliation

In [ ]:
def createAllPartyDicts(In_Climate_Dataframe,In_Fraction_Dataframe):
    Rep_Dict = createSinglePartyDict(In_Climate_Dataframe,In_Fraction_Dataframe,"Rep")
    Dem_Dict = createSinglePartyDict(In_Climate_Dataframe,In_Fraction_Dataframe,"Dem")
    Indep_Dict = createSinglePartyDict(In_Climate_Dataframe,In_Fraction_Dataframe,"Indep") 
    Dict_Dict = {"Rep_Dict":Rep_Dict,"Dem_Dict":Dem_Dict,"Indep_Dict":Indep_Dict}
    return(Dict_Dict)

### setup numpy matrices for news media reputations for a single political party ###

**Inputs: ** <br>
- Party_Affilication (string) - designates which party the matrices are being created for.  Affects the starting media reputation scores

**Outputs: ** <br>
- Rep_Pro_News (numpy matrix) - reputation of pro climate change news outlets for each state and game 
- Rep_Anti_News (numpy matrix) - reputation of anti climate change news outlets for each state and game

In [ ]:
def createNewsMatrices(Party_Affiliation):
    Rep_Anti_News = np.zeros(shape=(50,50))
    Rep_Pro_News = np.zeros(shape=(50,50))
    if(Party_Affiliation == "Rep"):   
        pro_news_start = np.ones(shape=(1,50))*-0.5
        anti_news_start = np.ones(shape=(1,50))*0.5
    elif(Party_Affiliation == "Dem"):
        pro_news_start = np.ones(shape=(1,50))*0.5
        anti_news_start = np.ones(shape=(1,50))*-0.5
    else:
        pro_news_start = np.ones(shape=(1,50))*0.1
        anti_news_start = np.ones(shape=(1,50))*0.1
    Rep_Anti_News[:,0] = anti_news_start
    Rep_Pro_News[:,0] = pro_news_start
    return([Rep_Pro_News,Rep_Anti_News])

### setup numpy matrices for a single opinion topic  for a single party ###

**Inputs: ** <br>
- Opinion (numpy matrix) - matrix to hold party opinions
- Party_Affiliation (string) - which party to create the opinion matrix for
- In_Climate_Dataframe (pandas dataframe) - Yale Climate Change data to initialize party opinions
- Outcome (string) - which outcome to add to the opinion matrix
- Outcome_Index (int) - the column in the Opinion matrix which corresponds to the current opinion to add

In [9]:
def createOpinionMatrix(Opinion,Party_Affiliation,In_Climate_Dataframe,Outcome,Outcome_Index):
    if(Party_Affiliation in ["Rep","Dem"]):
        Party_Subset = In_Climate_Dataframe.loc[In_Climate_Dataframe['Group'] == Party_Affiliation]
        Opinion[:,0,Outcome_Index] = (Party_Subset[Outcome]-50)/50.0
    else:
        Dem_Subset = In_Climate_Dataframe.loc[In_Climate_Dataframe['Group'] == "Dem"]
        Rep_Subset = In_Climate_Dataframe.loc[In_Climate_Dataframe['Group'] == "Rep"]
        Opinion[:,0,Outcome_Index] = ((np.add(Dem_Subset[Outcome],Rep_Subset[Outcome])/2.0)-50.0)/50.0

###  create and initiailize matrix to store opinions for all states for one political party ###

**Inputs: ** <br>
- Outcome_List (string array) - topics to add to the opinion matrix
- Party_Affiliation (string) - which party the outcome matrix is being created for 
- In_Climate_Dataframe (pandas dataframe) - dataframe containing Yale climate change survey data to populate the opinion matrix

**Outputs: ** <br>
- Party_Dict (dictionary) - dict containing the opinion matrix

In [ ]:
def createOutcomeMatrix(Outcome_List,Party_Affiliation,In_Climate_Dataframe):
    Opinion = np.zeros(shape=(50,50,len(Outcome_List)))
    Outcome_Index = 0
    for outcome in OUTCOME_LIST:
        createOpinionMatrix(Opinion,Party_Affiliation,In_Climate_Dataframe,outcome,Outcome_Index)
        Outcome_Index +=1
    Party_Dict = {'Opinion':Opinion}
    return(Party_Dict)

### create dictionary for a single party based on input data ###

**Inputs: ** <br>
- In_Climate_Dataframe (pandas dataframe) - dataframe containing Yale climate change survey data to populate the opinion matrix
- In_Fraction_Dataframe (pandas dataframe) - dataframe containing party affiliation for each state
- Party_Affiliation (string) - party affiliation

**Outputs: ** <br>
- Party_Dict (dictionary) - Party ditionary

In [ ]:
def createSinglePartyDict(In_Climate_Dataframe,In_Fraction_Dataframe,Party_Affiliation):
    Party_Dict = createOutcomeMatrix(OUTCOME_LIST,Party_Affiliation,In_Climate_Dataframe)
    [Rep_Pro_News,Rep_Anti_News] = createNewsMatrices(Party_Affiliation)
    Fraction = np.zeros(shape=(50,1))
    Fraction[:,0] = In_Fraction_Dataframe[Party_Affiliation]/100.0
    Resiliance = np.zeros(shape=(50,50,len(OUTCOME_LIST)))
    I_Agree_A1 = np.zeros(shape=(50,50,len(OUTCOME_LIST)))
    I_Agree_A2 = np.zeros(shape=(50,50,len(OUTCOME_LIST)))
    I_Media_Pro = np.zeros(shape=(50,50))
    I_Media_Anti = np.zeros(shape=(50,50))
    
    Party_Dict.update({'Fraction':Fraction,'Rep_Anti_News':Rep_Anti_News,
             'Rep_Pro_News': Rep_Pro_News, 'Resiliance':Resiliance, 'I_Agree_A1':I_Agree_A1,
             'I_Agree_A2':I_Agree_A2,'I_Media_Pro':I_Media_Pro,'I_Media_Anti':I_Media_Anti})
    return(Party_Dict)

### calculate Resiliance for a single political party ###

**Inputs: ** <br>
- Party_Dict (dictionary) - contains party specific game attributes
- Game_Index (int) - most recent game number

In [ ]:
def calcResilianceOneParty(Party_Dict,Game_Index,debug="None"):
    Opinion3DMatrix = Party_Dict['Opinion']
    #Resiliance = np.exp(-abs(Opinion3DMatrix[:,Game_Index,:]))
    #Resiliance = numpy.exp(-abs(Opinion[Curr_Index]))
    Party_Dict['Resiliance'][:,Game_Index,:] = np.exp(-abs(Opinion3DMatrix[:,Game_Index,:]))
    if(debug != "None"):
        print("Opinion")
        print(Opinion3DMatrix[25,Game_Index,:])
        print("TestResiliance")
        print(Party_Dict['Resiliance'][25,Game_Index,:])

### calculate Resiliance for all political parties ###

**Inputs: ** <br>
- Dict_Dict (dictionary) - contains subdictionaries for each political party
- Game_Index (int) - most recent game number

In [ ]:
def calcResilianceAllParties(Dict_Dict,Game_Index,debug="None"):
    calcResilianceOneParty(Dict_Dict['Rep_Dict'],Game_Index,debug)
    calcResilianceOneParty(Dict_Dict['Dem_Dict'],Game_Index,debug)
    calcResilianceOneParty(Dict_Dict['Indep_Dict'],Game_Index,debug)

### calculate Agree for an actor and one game ###

**Inputs: ** <br>
- Party_Dict (dictionary) - Dictionary storing attributes for a single political party
- Game_Index (int) - most recent game number
- Actor_Opinion (string) - whether actor agrees or disagrees with the actor
- Actor_Label (string) - label for current actor

In [ ]:
def calcAgreeOneParty(Party_Dict,Game_Index,Actor_Opinion,Actor_Label,debug="None"):
    Opinion3DMatrix = Party_Dict['Opinion']
    Party_Dict[Actor_Label][:,Game_Index,:] = np.exp(-abs(Actor_Opinion-Opinion3DMatrix[:,Game_Index,:] ))
    if(debug != "None"):
        print("Opinion")
        print(Opinion3DMatrix[25,Game_Index,:])
        print("IAgree")
        print(Party_Dict[Actor_Label][25,Game_Index,:])

### calculate Agree for an actor and one game and all parties ###

**Inputs: ** <br>
- Dict_Dict (dictionary) - Dictionary containing subdictionaries for each political party
- Game_Index (int) - most recent game number
- Actor_Opinion (string) - whether actor agrees or disagrees with the actor
- Actor_Label (string) - label for current actor

In [ ]:
def calcAgreeAllParties(Dict_Dict,Game_Index,Actor_Opinion,Actor_Label,debug="None"):
    calcAgreeOneParty(Dict_Dict['Rep_Dict'],Game_Index,Actor_Opinion,Actor_Label,debug)
    calcAgreeOneParty(Dict_Dict['Dem_Dict'],Game_Index,Actor_Opinion,Actor_Label,debug)
    calcAgreeOneParty(Dict_Dict['Indep_Dict'],Game_Index,Actor_Opinion,Actor_Label,debug)

### calculate Media Impact for one game and all political parties ###

**Inputs: ** <br>
- Dict_Dict (dictionary) - Dictionary containing subdictionaries for each political party
- Game_Index (int) - most recent game number
- Media_Rep_Label (string) - media type preferred by political party
- Media_Impact_Lebel (string) - media type (pro or anti climate chnage) to calculate impact for

In [ ]:
def calcMediaImpactAllParties(Dict_Dict,Game_Index,Media_Rep_Label,Media_Impact_Label,debug="None"):
    calcMediaImpactOneParty(Dict_Dict['Rep_Dict'],Game_Index,Media_Rep_Label,Media_Impact_Label,debug)
    calcMediaImpactOneParty(Dict_Dict['Dem_Dict'],Game_Index,Media_Rep_Label,Media_Impact_Label,debug)
    calcMediaImpactOneParty(Dict_Dict['Indep_Dict'],Game_Index,Media_Rep_Label,Media_Impact_Label,debug)

### calculate Media Impact for one game and one political party ###

**Inputs: ** <br>
- Party_Dict (dictionary) - Dictionary containing game model attributes for a specific political party
- Game_Index (int) - most recent game number
- Media_Rep_Label (string) - media type preferred by political party
- Media_Impact_Lebel (string) - media type (pro or anti climate chnage) to calculate impact for

In [ ]:
def calcMediaImpactOneParty(Party_Dict,Game_Index,Media_Rep_Label,Media_Impact_Label,debug="None"):
    Media_Rep = Party_Dict[Media_Rep_Label]
    Party_Dict[Media_Impact_Label][:,Game_Index] = np.power(Media_Rep[:,Game_Index],3)
    if(debug != "None"):
        print("Media Rep:")
        print(Media_Rep[1:5,Game_Index])
        print("Media Impact")
        print(Party_Dict[Media_Impact_Label][1:5,Game_Index])

### calculate change in media reputation for one media type and all political parties ###

**Inputs: ** <br>
- Dict_Dict (dictionary) - Dictionary containing subdictionaries for each political party
- Game_Index (int) - most recent game number
- Actor_Dict (dictionary) - dictionary containing attributes specific to each actor
- Actor_Party (string) - which political party is by default aligned with the actor

In [ ]:
def calcDeltaRepAllParties(Dict_Dict,Game_Index,Actor_Dict,Actor_Party):
    if(Actor_Party=='repub'):
        calcDeltaRepOneParty(Dict_Dict['Rep_Dict'],Game_Index,Actor_Dict,[-0.01,0.01])
    else:
        calcDeltaRepOneParty(Dict_Dict['Dem_Dict'],Game_Index,Actor_Dict,[0.01,-0.01])

### calculate change in media reputation for one media type and one political party ###

**Inputs: ** <br>
- Party_Dict (dictionary) - Dictionary containing game model attributes specific to a political party
- Game_Index (int) - most recent game number
- Actor_Dict (dictionary) - dictionary containing attributes specific to each actor
- Party_Drift (float) - natural drift in reputation toward political party affilitation

In [ ]:
def calcDeltaRepOneParty(Party_Dict,Game_Index,Actor_Dict,party_drift,debug="None"):
    expCalc = np.exp(Party_Dict['Rep_Pro_News'][:,Game_Index]*5)
    denominator = np.power(1+expCalc,2)
    DeltaReputation =np.divide(expCalc,denominator)
    DeltaReputation = np.multiply(DeltaReputation,Actor_Dict['ProCoop'][:,Game_Index])
    maxVals = np.ones(50)
    minVals = maxVals*-1
    maxVals2 = np.maximum(minVals,Party_Dict['Rep_Pro_News'][:,Game_Index] + DeltaReputation + party_drift[0] )
    minVals2 = np.minimum(maxVals2,maxVals)
    Party_Dict['Rep_Pro_News'][:,Game_Index+1] = minVals2
                                                                                           # + DeltaReputation + party_drift[0]))
    
    expCalc = np.exp(Party_Dict['Rep_Anti_News'][:,Game_Index]*5)
    denominator = np.power(1+expCalc,2)
    DeltaReputation =np.divide(expCalc,denominator)
    DeltaReputation = np.multiply(DeltaReputation,Actor_Dict['AntiCoop'][:,Game_Index])
    maxVals2 = np.maximum(minVals,Party_Dict['Rep_Anti_News'][:,Game_Index] + DeltaReputation + party_drift[1] )
    minVals2 = np.minimum(maxVals2,maxVals)
    Party_Dict['Rep_Anti_News'][:,Game_Index+1] = minVals2

    
    if(debug!="None"):
        print("Reputation: ")
        print(Party_Dict['Rep_Pro_News'][25,Game_Index])
        print("Delta Reputation: ")
        print(DeltaReputation[25])
        print("New Reputation: ")
        print(Party_Dict['Rep_Pro_News'][25,Game_Index+1])
    
    
        

### Calculate change in opinion for one political party ###

**Inputs: ** <br>
- Party_Dict (dictionary) - contains attributes specific to the political party
- Game_Index (int) - game number in a multi game sequence
- State_Index (int) - row corresponding to the current state of interest
- Actor_Aggre_Label (string) - attribute name for the attribute in the dict containing the I_Agree values
- Same_Party (boolean) - whether the actor represents the same political party as the current target population
- ACtor_Pos (String) - whether the actor is for ("Pro") or against ("Anti") climate change regulation

**Outputs: ** <br>
- Pro_Utility_Matrix - Utility function inputs based on pro media communications
- Anti_Utility_Matrix - Utility function inputs based on anti-media communications
- Total_Utility_Matrix - Sum of Pro and Anti Utility Matrices

In [ ]:
def calcDeltaOpinion_1Actor_OneParty(Party_Dict,Game_Index,State_Index,Actor_Agree_Label,Same_Party,Actor_Pos,debug="None"):
    Multiplier = np.multiply(Party_Dict['Resiliance'][State_Index,Game_Index,:],
                             Party_Dict[Actor_Agree_Label][State_Index,Game_Index,:])
    Pro_Utility_Matrix = calc1Actor_1NewsUtilityMatrix(Party_Dict,'I_Media_Pro',Multiplier,Same_Party,State_Index,Game_Index,
                                                       Actor_Pos,debug)
    Anti_Utility_Matrix = calc1Actor_1NewsUtilityMatrix(Party_Dict,'I_Media_Anti',Multiplier,Same_Party,State_Index,Game_Index,
                                                        Actor_Pos,debug)
   
    Total_Utility_Matrix = Anti_Utility_Matrix.T + Pro_Utility_Matrix
    Total_Utility_Matrix = np.reshape(Total_Utility_Matrix,(1,36))
    Pro_Utility_Matrix = np.reshape(Pro_Utility_Matrix,(1,36))
    Anti_Utility_Matrix = np.reshape(Anti_Utility_Matrix,(1,36))
    if(debug != "None"):
        print("Total Utility Matrix:")
        print(Total_Utility_Matrix)
    return([Pro_Utility_Matrix,Anti_Utility_Matrix,Total_Utility_Matrix])

### Calculate utility function scores for one news media outlet type, one actor, and one political party ###

**Inputs: ** <br>
- Party_Dict (dictionary) - contains attributes specific to the political party
- I_Media_Label (string) - whether media source is pro or anti climate change
- Multiplier (float) - learning rate for a single game
- Same_Party (boolean) - whether the actor represents the same political party as the current target population
- State_Index (int) - row corresponding to the current state of interest
- Game_Index (int) - game number
- Actor_Pos (string) -whether actor is pro or anti- climate change

**Outputs: ** <br>
- Utility_Mat (numpy matrix) - utility function scores for all strategies

In [ ]:
def calc1Actor_1NewsUtilityMatrix(Party_Dict,I_Media_Label,Multiplier,Same_Party,State_Index,Game_Index,Actor_Pos,debug="None"):
    
    Utility_Mat = np.ones(shape=(6,6))
    I_Media = Party_Dict[I_Media_Label][State_Index,Game_Index]
    if(Same_Party):
        I_Media = abs(I_Media)
    Temp_Mat = np.zeros(shape=(1,6))
    Temp_Mat[:,0:5] = np.multiply(Multiplier,I_Media)
    if(Actor_Pos == "Pro"):
        MaxUtility = np.zeros(shape=(1,6))
        MaxUtility[:,0:5] = 1-Party_Dict['Opinion'][State_Index,Game_Index,:]
        Temp_Mat = np.minimum(MaxUtility,Temp_Mat)
    else:
        MinUtility = np.zeros(shape=(1,6))
        MinUtility[:,0:5] =  np.abs(-1-Party_Dict['Opinion'][State_Index,Game_Index,:])
        Temp_Mat = np.minimum(MinUtility,Temp_Mat)
     
    Utility_Mat = np.multiply(Temp_Mat,Utility_Mat)
    
    if(debug!="None"):
        print("News_Mat: ")
        print(Utility_Mat)
    return(Utility_Mat)

### Calculate change in opinion for one actor and all parties ###

**Inputs: ** <br>
- Dict_Dict (dictionary) - dictionary with subdictionaries for each political party
- Game_Index (int) - game number
- State_Index (int) - row corresponding to the current state of interest
- Actor_Agree_Label (string) - whether actor and default political party position are in alignment

**Outputs: ** <br>
- Utility_Dict (dictionary) - utility function scores for all strategies

In [ ]:
def calcDeltaOpinion_1Actor_AllParties(Dict_Dict,Game_Index,State_Index,Actor_Agree_Label,debug="None"):
    if(Actor_Agree_Label=='I_Agree_A2'):
        [Rep_Pro_Utility, Rep_Anti_Utility,Rep_Utility] = calcDeltaOpinion_1Actor_OneParty(
            Dict_Dict['Rep_Dict'],Game_Index,State_Index,Actor_Agree_Label,True,"Anti",debug)
        [Dem_Pro_Utility, Dem_Anti_Utility,Dem_Utility] = calcDeltaOpinion_1Actor_OneParty(
            Dict_Dict['Dem_Dict'],Game_Index,State_Index,Actor_Agree_Label,False,"Anti",debug)
    else:
        [Rep_Pro_Utility, Rep_Anti_Utility, Rep_Utility] = calcDeltaOpinion_1Actor_OneParty(
            Dict_Dict['Rep_Dict'],Game_Index,State_Index,Actor_Agree_Label,False,"Pro",debug)
        [Dem_Pro_Utility, Dem_Anti_Utility, Dem_Utility] = calcDeltaOpinion_1Actor_OneParty(
            Dict_Dict['Dem_Dict'],Game_Index,State_Index,Actor_Agree_Label,True,"Pro",debug)
    [Indep_Pro_Utility,Indep_Anti_Utility,Indep_Utility] = calcDeltaOpinion_1Actor_OneParty(
        Dict_Dict['Indep_Dict'],Game_Index,State_Index,Actor_Agree_Label,True,"Pro",debug)
    Utility_Dict = {'Rep_Pro':Rep_Pro_Utility,'Rep_Anti':Rep_Anti_Utility,'Rep_Utility':Rep_Utility,
                    'Dem_Pro':Dem_Pro_Utility,'Dem_Anti':Dem_Anti_Utility,'Dem_Utility':Dem_Utility,
                    'Indep_Pro':Indep_Pro_Utility,'Indep_Anti':Indep_Anti_Utility,'Indep_Utility':Indep_Utility}
    return(Utility_Dict)

### Calculate utility matrix for one actor and all political parties ###

**Inputs: ** <br>
- Dict_Dict (dictionary) - dictionary with subdictionaries for each political party
- Delta_Opionion_Dict (dictionary) - dictionary containing utility matrices for all political parties
- Game_Index (int) - game number
- State_Index (int) - row corresponding to the current state of interest
- Multiplier (float) - learning rate 

**Outputs: ** <br>
- Total_Weighted_Utility (numpy matrixx) - Utility function scores for a single actor

In [ ]:
def calcActorUtilityMatrix(Dict_Dict,Delta_Opinion_Dict,Game_Index,State_Index,Multiplier,debug="None"):
    Republican_Utility = Dict_Dict['Rep_Dict']['Fraction'][State_Index]*Delta_Opinion_Dict['Rep_Utility']
    Dem_Utility = Dict_Dict['Dem_Dict']['Fraction'][State_Index]*Delta_Opinion_Dict['Dem_Utility']
    Indep_Utility = Dict_Dict['Indep_Dict']['Fraction'][State_Index]*Delta_Opinion_Dict['Indep_Utility']
    Total_Weighted_Utility = Republican_Utility + Dem_Utility + Indep_Utility
    Total_Weighted_Utility *=Multiplier
    Total_Expanded_Utility = np.ones(shape=(36,36))
    Total_Expanded_Utility = np.multiply(Total_Weighted_Utility,Total_Expanded_Utility)
    
    if(debug != "None"):
        print("Total Weighted Utility: ")
        print(Total_Weighted_Utility)
        print("Total Expanded Utility: ")
        print(Total_Expanded_Utility)
    return(Total_Weighted_Utility)

### Calculate Payoff matrix by calculating and summing utility functions for both actors ###

**Inputs: ** <br>
- Dict_Dict (dictionary) - dictionary with subdictionaries for each political party
- A1_Delta_Opionion_Dict (dictionary) - change in opinion based on Actor 1 strategies
- A2_Delta_Opionion_Dict (dictionary) - change in opinion based on Actor 1 strategies
- Game_Index (int) - game number
- State_Index (int) - row corresponding to the current state of interest

**Outputs: ** <br>
- Payoff_Matrix (numpy matrix) - payoff matrix 

In [ ]:
def calcPayoffMatrix(Dict_Dict,A1_Delta_Opinion_Dict,A2_Delta_Opinion_Dict,Game_Index,State_Index,debug="None"):
    Total_A1_Utility = calcActorUtilityMatrix(Dict_Dict,A1_Delta_Opinion_Dict,Game_Index,State_Index,1,debug)
    Total_A2_Utility = calcActorUtilityMatrix(Dict_Dict,A2_Delta_Opinion_Dict,Game_Index,State_Index,-1,debug)
    Payoff_Matrix = Total_A1_Utility + Total_A2_Utility.T
    
    return(Payoff_Matrix)

### Determine whether a strategy is dominated ###

**Inputs: ** <br>
- Payoff_Matrixx(numpy matrix) - payoff matrix
- index (int) - strategy to determine if it is dominated

**Outputs: ** <br>
- Worst_Score (float) - return worst among all potential strategies

In [ ]:
def Is_Dominated(Payoff_Matrix,index,row=True):
    Worst_Score = None
    AxisVal = 1
    if(row):
        AxisVal = 0
    if(row):
        Strat_To_Test = Payoff_Matrix[index]
        Worst_Score = 0
    else:
        Strat_To_Test = Payoff_Matrix[:,index]
        Worst_Score = 0
    for Compare_Index in range(0,Payoff_Matrix.shape[AxisVal]):
        if(Compare_Index != index):
            if(row):
                Compare_Strat = Payoff_Matrix[Compare_Index]
                if(np.alltrue(Compare_Strat < Strat_To_Test)):
                    Worst_Score = min(sum(np.subtract(Compare_Strat, Strat_To_Test)),Worst_Score)
            else:
                Compare_Strat = Payoff_Matrix[:,Compare_Index]
                if(np.alltrue(Compare_Strat > Strat_To_Test)):
                    Worst_Score = max(sum(np.subtract(Compare_Strat,Strat_To_Test)), Worst_Score)
    return Worst_Score
    
    

### update change in opinion for one political party ###

**Inputs: ** <br>
- Party_Dict (dictionary) - dictionary with unique characteristics for each political party
- A1_Move (dictionary) - move selected by actor 1
- A2_Move (dictionary) - move selected by actor 2
- A1_Party_DOpinion (numpy array) - change in party opinion based on actor 1 actions
- A2_Party_DOpinion (numpy array) - change in party opinion based on actor 2 actions
- Party (string) - which party's opinion is being updated
- Game_Index (int) - game number
- State_Index (float) - state index currently being analyzed

In [ ]:
def updateOpinionsOneParty(Party_Dict,A1_Move,A2_Move,A1_Party_DOpinion,A2_Party_DOpinion,Party,Game_Index,State_Index):
    A1 = [0,0]
    A2 = [0,0]
    if(Party=="Rep"):
        A1[0] = float(A1_Party_DOpinion['Rep_Pro'][0,A1_Move])
        A1[1] = float(A1_Party_DOpinion['Rep_Anti'][0,A1_Move])
        A2[0] = float(A2_Party_DOpinion['Rep_Pro'][0,A2_Move])*-1.0
        A2[1] = float(A2_Party_DOpinion['Rep_Anti'][0,A2_Move])*-1.0
    elif(Party=="Dem"):
        A1[0] = float(A1_Party_DOpinion['Dem_Pro'][0,A1_Move])
        A1[1] = float(A1_Party_DOpinion['Dem_Anti'][0,A1_Move])
        A2[0] = float(A2_Party_DOpinion['Dem_Pro'][0,A2_Move])*-1.0
        A2[1] = float(A2_Party_DOpinion['Dem_Anti'][0,A2_Move])*-1.0
    else:
        A1[0] = float(A1_Party_DOpinion['Indep_Pro'][0,A1_Move])
        A1[1] = float(A1_Party_DOpinion['Indep_Anti'][0,A1_Move])
        A2[0] = float(A2_Party_DOpinion['Indep_Pro'][0,A2_Move])*-1.0
        A2[1] = float(A2_Party_DOpinion['Indep_Anti'][0,A2_Move])*-1.0
    addVals(Party_Dict,State_Index,Game_Index,A1,A1_Move,True)
    addVals(Party_Dict,State_Index,Game_Index,A2,A2_Move,False)

### update opinion scores based on actor actions and consequent utilities ###

**Inputs: ** <br>
- Party_Dict (dictionary) - dictionary containing attributes unique to each political party
- State_Index (int) - row corresponding to the current state of interest
- Game_Index (int) - game number
- Vals_To_Add (numpy array) - change in opinion score for all states
- Move (integer) - move selected by actor
- Is_New_State (boolean) - whether values to add to the state is a new state entry in the dataset
- A1_Delta_Opionion_Dict (dictionary) - change in opinion based on Actor 1 strategies
- A2_Delta_Opionion_Dict (dictionary) - change in opinion based on Actor 1 strategies

In [ ]:
def addVals(Party_Dict,State_Index,Game_Index,Vals_To_Add,Move,Is_New_State):
    Learning_Rate = 1
    for Outcome_Index in range(0,len(OUTCOME_LIST)):
        AmtToAdd = 0
        if(Move % len(OUTCOME_LIST) == 0):
            AmtToAdd = Vals_To_Add[0]*Learning_Rate
        elif(Vals_To_Add[0] != 0):
            AmtToAdd += (0.0001 + Vals_To_Add[0]*Learning_Rate) /2.0
        if(int(Move/len(OUTCOME_LIST)) == Outcome_Index):
            AmtToAdd += Vals_To_Add[1]*Learning_Rate
        elif(Vals_To_Add[1] != 0):
            AmtToAdd += (0.0001 + Vals_To_Add[1]*Learning_Rate)/2.0
        if(Is_New_State):
            Temp_Vals = np.maximum(-1.0,np.minimum(
                Party_Dict['Opinion'][State_Index,Game_Index,Outcome_Index] + AmtToAdd,1.0))
        else:
            Temp_Vals = np.maximum(-1.0,np.minimum(Party_Dict['Opinion'][State_Index,Game_Index+1,Outcome_Index] + AmtToAdd, 1.0))
        Party_Dict['Opinion'][State_Index,Game_Index+1,Outcome_Index] = Temp_Vals

### update change in opinion scores for all political parties ###

**Inputs: ** <br>
- Dict_Dict (dictionary) - dictionary with subdictionaries for each political party
- A1_Move (int) - strategy selected by actor 1
- A2_Move (int) - strategy selected by actor 2
- A1_Opinion_Dict (dictionary) - Change in opinions caused by actor1 actions
- A2_Opinion_Dict (dictionary) - Change in opinions caused by actor2 actions
- Game_Index (int) - game number
- State_Index (int) - row corresponding to the current state of interest

**Outputs: ** <br>
- Payoff_Matrix (numpy matrix) - payoff matrix 

In [ ]:
def updateOpinionScoresAllParties(Dict_Dict,A1_Move,A2_Move,A1_Opinion_Dict,A2_Opinion_Dict,Game_Index,State_Index):
    updateOpinionsOneParty(Dict_Dict['Rep_Dict'],A1_Move,A2_Move,A1_Opinion_Dict,
                           A2_Opinion_Dict,"Rep",Game_Index,State_Index)
    updateOpinionsOneParty(Dict_Dict['Dem_Dict'],A1_Move,A2_Move,A1_Opinion_Dict,
                           A2_Opinion_Dict,"Dem",Game_Index,State_Index)
    updateOpinionsOneParty(Dict_Dict['Indep_Dict'],A1_Move,A2_Move,A1_Opinion_Dict,
                            A2_Opinion_Dict,"Indep",Game_Index,State_Index)
    

### find dominated strategies  ###

**Inputs: ** <br>
- Payoff_Matrix (numpy_array) - payoff matrix

**Outputs: ** <br>
- Worst_Index (integer) - the domainted strategy (if there is one) with the worst score

In [ ]:
def Find_Dominated(Payoff_Matrix,row=True):
    if(row): 
        axis = 0 
    else: 
        axis = 1
    Worst_Score = [0]*Payoff_Matrix.shape[axis]
    for Strategy_Index in range(0,Payoff_Matrix.shape[axis]):
        Worst_Score[Strategy_Index] = abs(Is_Dominated(Payoff_Matrix,Strategy_Index,row))
        #print("Worst Score for Strategy " + str(Strategy_Index) + ": " +
        #      str(Worst_Score[Strategy_Index]))
    Worst_Index = Worst_Score.index(max(Worst_Score))
    if(Worst_Score[Worst_Index] == 0):
        #print("No Dominated Strategy")
        return None
    return(Worst_Index)
        

### perform backward induction to identify optimal strategies ###

**Inputs: ** <br>
- Payoff_Matrix (numpy array) - payoff matrix 

**Outputs: ** <br>
- Payoff_Reduced (numpy matrix) - payoff matrix reduced to optimal strategy(ies)
- Candidate_Rows (numpy array) - optimal actor 2 strategies
- Candidate_Cols (numpy array) - optimal actor 1 strategies

In [ ]:
def Backward_Induction(Payoff_Matrix):
    Payoff_Reduced = deepcopy(Payoff_Matrix)
    Curr_Player_Row = True
    Curr_Axis = 0
    Candidate_Rows = range(0,36)
    Candidate_Cols = range(0,36)
    No_More_Dominated = 0
    while(No_More_Dominated < 2):
        Worst_Index = Find_Dominated(Payoff_Reduced,Curr_Player_Row)
        if(Worst_Index != None):# and Payoff_Reduced.shape[Curr_Axis]>2):
            Payoff_Reduced = np.delete(Payoff_Reduced, [Worst_Index], Curr_Axis)
            if(Curr_Player_Row):
                del(Candidate_Rows[Worst_Index])
            else:
                del(Candidate_Cols[Worst_Index])
            No_More_Dominated = 0
        else:
            No_More_Dominated +=1
        Curr_Player_Row = not Curr_Player_Row
        Curr_Axis = (Curr_Axis+1) % 2
    return([Payoff_Reduced,Candidate_Rows,Candidate_Cols])


### if there's no dominant strategy, randomly select among optimal strategies ###

**Inputs: ** <br>
- In_Row (numpy array) - integer array cotaning multiple actor 1 actions
- In_Col (numpy array) - integer array cotaning multiple actor 1 actions

**Outputs: ** <br>
- Row (int) - randomly select actor 2 move
- Col (int) - randomly selected actor 1 move

In [ ]:
def randomSelection(In_Row,In_Col):
    Row = np.random.choice(In_Row, 1)
    Col = np.random.choice(In_Col, 1)
    return([Row,Col])

### initialize game attributes ###

In [ ]:
def setupGame():
    Dict_Dict = createAllPartyDicts(Climate_Support,Party_Fractions)
    Actor1_Moves = np.zeros(shape=(50,50))
    Actor1_Pro_Coop = np.zeros(shape=(50,50))
    Actor1_Anti_Coop = np.zeros(shape=(50,50))
    Actor1_Dict = {'Moves':Actor1_Moves,'ProCoop':Actor1_Pro_Coop,'AntiCoop':Actor1_Anti_Coop}
    Actor2_Dict = deepcopy(Actor1_Dict)
    return([Dict_Dict,Actor1_Dict,Actor2_Dict])

### update acctor dictionary with most recent actions ###

**Inputs: ** <br>
- Actor1 (dictionary) - dictionary containing attributes specific to actor 1
- Actor2 (dictionary) - dictionary containing attributes specific to actor 2
- Row (int) - row in payoff matrix corresponding to actor 2 choice
- Col (int) - column in payoff matrix corresponding to actor 2 choice
- State_Index (int) - row number tht corresponds to the current state
- Game_Index - game number

In [ ]:
def UpdateActorDicts(Actor1_Dict,Actor2_Dict,Row,Col,State_Index,Game_Index):
    Actor1['Moves'][State_Index,Game_Index] = int(Col[0])
    Actor2['Moves'][State_Index,Game_Index] = int(Row[0])
    if(Col < 30):
        Actor1['AntiCoop'][State_Index,Game_Index] = 1
    else:
        Actor1['AntiCoop'][State_Index,Game_Index] = -1 
    if(Col %6 != 0 ):
        Actor1['ProCoop'][State_Index,Game_Index] = 1
    else:
        Actor1['ProCoop'][State_Index,Game_Index] = -1
    if(Row < 30):
        Actor2['AntiCoop'][State_Index,Game_Index] = 1
    else:
        Actor2['AntiCoop'][State_Index,Game_Index] = -1 
    if(Row %6 != 0 ):
        Actor2['ProCoop'][State_Index,Game_Index] = 1
    else:
        Actor2['ProCoop'][State_Index,Game_Index] = -1

### calculate game theory components that can't be vectorized ###

**Inputs: ** <br>
- Dict_Dict (dictionary) - dictionary with subdictionaries for each political party
- Game_Index (int) - game number
- State_Index (int) - row number that corresponds to the current state 
- Actor1 (dictionary) - dictionary containing attributes specific to actor 1
- Actor2 (dictionary) - dictionary containing attributes specific to actor 2

In [ ]:
def runSingleState(Dict_Dict,Game_Index,State_Index,Actor1,Actor2):
    for State_Index in range(0,50):
        A1_Delta_Opinion_Dict = calcDeltaOpinion_1Actor_AllParties(Dict_Dict,Game_Index,State_Index,'I_Agree_A1')
        A2_Delta_Opinion_Dict = calcDeltaOpinion_1Actor_AllParties(Dict_Dict,Game_Index,State_Index,'I_Agree_A2')
        Payoff_Matrix = calcPayoffMatrix(Dict_Dict,A1_Delta_Opinion_Dict,A2_Delta_Opinion_Dict,Game_Index,State_Index)
        [Payoff_Reduced, Row,Col] = Backward_Induction(Payoff_Matrix)
        [Row,Col] = randomSelection(Row,Col)
        UpdateActorDicts(Actor1,Actor2,Row,Col,State_Index,Game_Index)
        updateOpinionScoresAllParties(Dict_Dict,Col,Row,A1_Delta_Opinion_Dict,A2_Delta_Opinion_Dict,
                                      Game_Index,State_Index)

### run a single game for a single state ###

**Inputs: ** <br>
- Dict_Dict (dictionary) - dictionary with subdictionaries for each political party
- Actor1 (dictionary) - dictionary containing attributes specific to actor 1
- Actor2 (dictionary) - dictionary containing attributes specific to actor 2

In [ ]:
def runSingleGame(Dict_Dict,Actor1,Actor2,Game_Index=0):
    calcResilianceAllParties(Dict_Dict,Game_Index)
    calcAgreeAllParties(Dict_Dict,Game_Index,1,"I_Agree_A1")
    calcAgreeAllParties(Dict_Dict,Game_Index,-1,"I_Agree_A2")
    calcMediaImpactAllParties(Dict_Dict,Game_Index,"Rep_Pro_News","I_Media_Pro")
    calcMediaImpactAllParties(Dict_Dict,Game_Index,"Rep_Anti_News","I_Media_Anti")
    runSingleState(Dict_Dict,Game_Index,25,Actor1,Actor2)
    calcDeltaRepAllParties(Dict_Dict,Game_Index,Actor1,"Dem")
    calcDeltaRepAllParties(Dict_Dict,Game_Index,Actor1,"repub")

### retreospectively calculate which moves an actor chose for each state ###

**Inputs: ** <br>
- Actor_Dict (dictionary) - dictionary containing attributes specific to a player
- outputFile (string) - complete filepath to location to write results in a csv file

In [ ]:
def calcActorMoves(Actor_Dict,outputFile):
    sep = ','
    new_line = '\n'
    fileWriter = open(outputFile,'w') 
    fileWriter.write("aReg,aRenew,aHappen,aHuman,aHarm,aNone,pReg,pRenew,pHappen,pHuman,pHarm,pNone \n")
    for State_Index in range(0,50):
        fileWriter.write(str(sum(Actor_Dict['Moves'][State_Index,0:25] < 6)) + sep)
        fileWriter.write(str(sum((5 < Actor_Dict['Moves'][State_Index,0:25]) \
                             * (Actor_Dict['Moves'][State_Index,0:25] < 12))) + sep)
        fileWriter.write(str(sum((11 < Actor_Dict['Moves'][State_Index,0:25] \
                             * (Actor_Dict['Moves'][State_Index,0:25] < 18)))) + sep)
        fileWriter.write(str(sum((17 < Actor_Dict['Moves'][State_Index,0:25]) \
                                 * (Actor_Dict['Moves'][State_Index,0:25] < 24))) + sep)
        fileWriter.write(str(sum((23 < Actor_Dict['Moves'][State_Index,0:25]) \
                                 * (Actor_Dict['Moves'][State_Index,0:25] < 30))) + sep)
        fileWriter.write(str(sum((29 < Actor_Dict['Moves'][State_Index,0:25]))) + sep)
        fileWriter.write(str(sum(Actor_Dict['Moves'][State_Index,0:25] %6 == 0)) + sep)
        fileWriter.write(str(sum(Actor_Dict['Moves'][State_Index,0:25] %6 == 1)) + sep)
        fileWriter.write(str(sum(Actor_Dict['Moves'][State_Index,0:25] %6 == 2)) + sep)
        fileWriter.write(str(sum(Actor_Dict['Moves'][State_Index,0:25] %6 == 3)) + sep)
        fileWriter.write(str(sum(Actor_Dict['Moves'][State_Index,0:25] %6 == 4)) + sep)
        fileWriter.write(str(sum(Actor_Dict['Moves'][State_Index,0:25] %6 == 5)) + sep)
        fileWriter.write("\n")            

### main function ###

In [ ]:
Dict_Dict = createAllPartyDicts(Climate_Support,Party_Fractions)
[Dict_Dict,Actor1,Actor2] = setupGame()
for Game_Index in range(0,3):
    runSingleGame(Dict_Dict,Actor1,Actor2,Game_Index)
Dict_Dict['Rep_Dict']['Rep_Anti_News'][:,0]
Payoff_Choice=(Backward_Induction(Dominated_Payoff))    
DeltaDemOpinion = Dict_Dict['Dem_Dict']['Opinion'][:,24,:] - Dict_Dict['Dem_Dict']['Opinion'][:,0,:]
DeltaRepOpinion = Dict_Dict['Rep_Dict']['Opinion'][:,24,:] - Dict_Dict['Rep_Dict']['Opinion'][:,0,:]

np.savetxt(Parent_Folder + 'DemDeltaOpinion.csv', DeltaDemOpinion, delimiter=',')
np.savetxt(Parent_Folder + 'RepDeltaOpinion.csv', DeltaRepOpinion, delimiter=',')

calcActorMoves(Actor2,Parent_Folder + "Actor1.csv")
calcActorMoves(Actor2,Parent_Folder + "Actor2.csv")


np.savetxt(Parent_Folder + 'HawaiiReps.csv', Dict_Dict['Rep_Dict']['Opinion'][10,:,:], delimiter=',')
np.savetxt(Parent_Folder + 'HawaiiDems.csv', Dict_Dict['Dem_Dict']['Opinion'][10,:,:], delimiter=',')
np.savetxt(Parent_Folder + 'MontanaReps.csv', Dict_Dict['Rep_Dict']['Opinion'][25,:,:], delimiter=',')
np.savetxt(Parent_Folder + 'MontanaDems.csv', Dict_Dict['Dem_Dict']['Opinion'][25,:,:], delimiter=',')